In [7]:
!pip install -q -U google-generativeai

In [ ]:
from google.colab import userdata
import google.generativeai as genai
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Plate Number Recognition From an Image

import google.generativeai as genai
import httpx
import base64
from IPython.display import Markdown
import os

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

def process_image_input(image_input):
    """
    Takes either an image URL or an image file path and returns the base64 encoded image data.
    """
    if isinstance(image_input, str) and image_input.startswith("http"):
        # If the input is a URL, fetch the image
        response = httpx.get(image_input)
        if response.status_code == 200:
            return base64.b64encode(response.content).decode("utf-8")
        else:
            raise ValueError("Failed to retrieve image from URL")

    elif os.path.isfile(image_input):
        # If the input is a file, read the image file
        with open(image_input, "rb") as file:
            return base64.b64encode(file.read()).decode("utf-8")

    else:
        raise ValueError("Invalid input: must be either a file path or a valid image URL.")

# Example of using either a URL or a file:
image_input = "https://upload.wikimedia.org/wikipedia/commons/thumb/2/2c/Indonesian_government%2C_military%2C_or_police_officials_license_plate.jpg/640px-Indonesian_government%2C_military%2C_or_police_officials_license_plate.jpg"
# Or if you want to use a local file:
# image_input = "media/images/car.jpg" # Put here the path to your image

# Process the image input to get base64-encoded data
image_data = process_image_input(image_input)

prompt = """
Please analyze the image of a vehicle's license plate and output the following details in JSON format:
{
    'plate': 'B 1234 ABC',
    'vehicle': 'car',
    'type': 'sedan',
    'color': 'red',
    'expiry_date': '07 - 17',
    'gate_open_time': '2024-12-02 18:15:01',
    'gate_closed_time': 'N/A'
}. Only return the JSON object, and use 'N/A' for missing or unknown information.
"""


# Send the request to the model
response = model.generate_content(
    [
        {
            "mime_type": "image/jpeg",
            "data": image_data,
        },
        prompt,
    ]
)

# Display the response in markdown
Markdown(response.text)


```json
{
    'plate': 'B 1308 RFK',
    'vehicle': 'car',
    'type': 'SUV',
    'color': 'black',
    'expiry_date': '09-22',
    'gate_open_time': 'N/A',
    'gate_closed_time': 'N/A'
}
```